In [1]:
!pip install chess

In [2]:
import chess.pgn

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_validate

In [4]:
def parse_pgn_game(game):
    # Extract relevant metadata
    metadata = {
        "Event": game.headers.get('Event', ""),
        "Site": game.headers.get('Site', ""),
        "Date": game.headers.get('Date', ""),
        "White": game.headers.get('White', ""),
        "Black": game.headers.get('Black', ""),
        "Result": game.headers.get('Result', ""),
        "WhiteElo": game.headers.get('WhiteElo', 0),
        "BlackElo": game.headers.get('BlackElo', 0),
        "TimeControl": game.headers.get('TimeControl', ""),
        "Termination": game.headers.get('Termination', "")
    }

    # Check if any essential metadata is missing
    if "?" in [metadata["White"], metadata["Black"]] or "?" in str(metadata["Date"]) or "????." in str(metadata["Date"]):
        return None, None, None, None

    # Initialize variables to store first clock times for white and black
    white_first_clock = None
    black_first_clock = None

    # Extract moves and evaluations
    moves = []
    clocks = []
    evaluations = []
    for node in game.mainline():
        move = node.move
        moves.append(str(move))
        move_data = node.comment
        if move_data:
            move_data_parts = move_data.split(' ')
            if len(move_data_parts) >= 4:  # Check if move_data has enough elements
                clock = move_data_parts[3].strip(']')
                evaluation = move_data_parts[1].strip(']')
                # Handle mate scores
                if evaluation.startswith('#'):
                    evaluation = np.sign(float(evaluation.replace('#', ''))) * float('inf')
                else:
                    evaluation = float(evaluation)
            else:
                evaluation = None
                clock = None
        else:
            evaluation = None
            clock = None
        evaluations.append(evaluation)
        clocks.append(clock)
        
        # Check and store the clock times for the first move of each player
        if white_first_clock is None and node.board().turn:
            white_first_clock = clock
        elif black_first_clock is None and not node.board().turn:
            black_first_clock = clock

            # If the first clock times of white and black are not the same, exclude this game
    if white_first_clock != black_first_clock:
        return None, None, None, None

    return metadata, moves, clocks, evaluations


In [5]:
import io  # Import io module

def preprocess_pgn(pgn_file):
    game_data = []
    current_game_lines = []

    with open(pgn_file) as f:
        for line in f:
            line = line.strip()
            if line.startswith("[Event"):
                if current_game_lines:
                    game = "\n".join(current_game_lines)
                    parsed_game = chess.pgn.read_game(io.StringIO(game))
                    if parsed_game:
                        metadata, moves, clocks, evaluations = parse_pgn_game(parsed_game)
                        if metadata is not None:
                            game_data.append((metadata, moves, clocks, evaluations))
                current_game_lines = []  # Reset for the next game
            current_game_lines.append(line)

        # Process the last game if it exists
        if current_game_lines:
            game = "\n".join(current_game_lines)
            parsed_game = chess.pgn.read_game(io.StringIO(game))
            if parsed_game:
                metadata, moves, clocks, evaluations = parse_pgn_game(parsed_game)
                if metadata is not None:
                    game_data.append((metadata, moves, clocks, evaluations))

    return game_data


In [6]:
import pandas as pd

def create_features_dataframe(game_data):
    dfs = []
    for game_id, (metadata, moves, clocks, evaluations) in enumerate(game_data):
        if metadata is None:
            continue  # Skip this game if metadata is missing

        # Extract metadata features
        metadata_features = [
            game_id,  # This is the unique identifier for each game
            metadata["WhiteElo"],
            metadata["BlackElo"],
            metadata["TimeControl"],
            metadata["Result"]
        ]

        # Extract move features (e.g., move count)
        move_count = len(moves)

        # Combine all features
        game_features = metadata_features + [move_count]

        # Create DataFrame for current game
        game_df = pd.DataFrame(columns=[
            "GameID", "WhiteElo", "BlackElo", "TimeControl", "Result", "MoveCount", "Move", "Turn", "Evaluation", "Clock"
        ])

        # Append move, turn, and evaluation for each move separately
        for i, move in enumerate(moves):
            move_features = game_features + [
                move, 
                'White' if i % 2 == 0 else 'Black', 
                evaluations[i] if evaluations else None, 
                clocks[i]
            ]  # Add move, turn, and evaluation for the current move
            game_df.loc[i] = move_features

        dfs.append(game_df)

    # Concatenate DataFrames for all games
    df = pd.concat(dfs, ignore_index=True)

    return df

In [7]:
# Define a function to preprocess the data
def preprocess_data(df):
    # Replace NaNs and infinities
    df.fillna(0, inplace=True)
    df.replace([np.inf, -np.inf], [100, -100], inplace=True)
    
    # Convert ELO ratings to numeric and drop rows with missing ratings
    df['WhiteElo'] = pd.to_numeric(df['WhiteElo'], errors='coerce')
    df['BlackElo'] = pd.to_numeric(df['BlackElo'], errors='coerce')
    df.dropna(subset=['WhiteElo', 'BlackElo'], inplace=True)

    # Calculate time-related features
    df['Clock'] = pd.to_timedelta(df['Clock'])
    if 'GameID' in df.columns:
        df['TimeSpent'] = df.groupby(['GameID'])['Clock'].diff().dt.total_seconds().fillna(0)

    # Extract time control data
    time_control_df = df['TimeControl'].str.extract(r'(?P<InitialTime>\d+)(?:\+(?P<Increment>\d+))?')
    time_control_df = time_control_df.fillna(0).astype(float)
    df['InitialTime'] = time_control_df['InitialTime']
    df['Increment'] = time_control_df['Increment']

    # Add time pressure feature
    time_pressure_threshold = 10  # Define the time pressure threshold
    df['TimePressure'] = df['Clock'].dt.total_seconds().lt(time_pressure_threshold).astype(int)

    # Calculate the absolute rating difference
    df["RatingDifference"] = abs(df["WhiteElo"] - df["BlackElo"])

    return df

In [8]:
# Example usage
pgn_file_path = "data/evals.pgn"  # Replace with the path to your PGN file
game_data = preprocess_pgn(pgn_file_path)
df = create_features_dataframe(game_data)

# Print the first few rows of the DataFrame
print(df.head())

len(df)

df = preprocess_data(df)

len(df)

FileNotFoundError: [Errno 2] No such file or directory: 'data/evals.pgn'

In [ ]:
# Make sure to update the features DataFrame after you've added the new columns
features = df.drop(columns=["WhiteElo", "BlackElo", "RatingDifference", "TimeControl", "Clock"])  # Assuming you want to exclude TimeControl and Clock from the features
target = df["RatingDifference"]

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
# Define preprocessing steps for numerical and categorical features
numeric_features = features.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = ["Turn"]  # Update with your categorical features
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ])

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# Define a new RandomForestRegressor with hyperparameters that limit complexity
model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)

# If you want to fine-tune hyperparameters further, consider using GridSearchCV or RandomizedSearchCV
parameters = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [5, 10, 15],
    # Add more parameters here
}

# Create a new pipeline with the simplified model
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

n_splits = 5

# Define the cross-validation iterator
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Set up the grid search
grid_search = GridSearchCV(pipeline, parameters, cv=kf, scoring='neg_mean_squared_error')

# Train the model
grid_search.fit(X_train, y_train)

# The best model found by the grid search
best_model = grid_search.best_estimator_

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Model Performance:\nMSE: {mse}\nMAE: {mae}\nR2: {r2}")


In [ ]:
display(df)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([min(y_test.values.min(), y_pred.min()), max(y_test.values.max(), y_pred.max())], 
         [min(y_test.values.min(), y_pred.min()), max(y_test.values.max(), y_pred.max())], 
         color='red', linestyle='--')
plt.title('Actual vs. Predicted Absolute Rating Differences')
plt.xlabel('Actual Rating Differences')
plt.ylabel('Predicted Rating Differences')
plt.grid(True)
plt.show()


In [ ]:
# Example usage
pgn_file_path = "data/evals_bg.pgn"  # Replace with the path to your PGN file
test_data = preprocess_pgn(pgn_file_path)
test_df = create_features_dataframe(test_data)

# Print the first few rows of the DataFrame
print(test_df.head())

len(test_df)

new_data = preprocess_data(test_df)

len(df)

In [ ]:
# Prepare the features dataset and make predictions
new_features = new_data.drop(columns=["WhiteElo", "BlackElo", "RatingDifference", "TimeControl", "Clock"])
new_predictions = pipeline.predict(new_features)

In [ ]:
# The new_predictions array now holds the predicted rating differences for the new data
new_data['PredictedRatingDifference'] = new_predictions

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming new_data contains 'PredictedRatingDifference' and 'RatingDifference'
# Summary statistics of the predictions
prediction_summary = new_data['PredictedRatingDifference'].describe()
print(prediction_summary)

# Plot the distribution of predicted rating differences
plt.figure(figsize=(10, 6))
sns.histplot(new_data['PredictedRatingDifference'], kde=True, bins=30)
plt.title('Distribution of Predicted Rating Differences')
plt.xlabel('Predicted Rating Difference')
plt.ylabel('Frequency')
plt.show()

# If you have actual rating differences for the new dataset, you can also visualize the predictions vs actuals
if 'RatingDifference' in new_data.columns:
    plt.figure(figsize=(10, 6))
    plt.scatter(new_data['RatingDifference'], new_data['PredictedRatingDifference'], alpha=0.5)
    plt.plot([new_data['RatingDifference'].min(), new_data['RatingDifference'].max()],
             [new_data['RatingDifference'].min(), new_data['RatingDifference'].max()],
             'k--', lw=2, color='red')  # Diagonal line for reference
    plt.title('Actual vs. Predicted Rating Differences')
    plt.xlabel('Actual Rating Difference')
    plt.ylabel('Predicted Rating Difference')
    plt.show()

    # Calculate and plot residuals
    new_data['Residuals'] = new_data['RatingDifference'] - new_data['PredictedRatingDifference']
    plt.figure(figsize=(10, 6))
    sns.histplot(new_data['Residuals'], kde=True, bins=30)
    plt.title('Residuals of Predictions')
    plt.xlabel('Residual (Actual - Predicted)')
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
# Assuming 'new_data' is your DataFrame and it has a 'GameID' column
# Keep only the game-level metadata and the predicted/actual rating differences
game_level_data = new_data.groupby('GameID').agg({
    'PredictedRatingDifference': 'mean',  # Taking the mean predicted difference per game
    'RatingDifference': 'first',  # Assuming actual rating difference is the same for all moves in a game
    # Include any other metadata columns you want to keep, like:
    'WhiteElo': 'first',
    'BlackElo': 'first',
    'TimeControl': 'first',
    'Result': 'first',
    'MoveCount': 'first',
    
    # ... etc.
}).reset_index()

# Now let's create the visualizations using 'game_level_data'

# Plot the distribution of predicted rating differences
plt.figure(figsize=(10, 6))
sns.histplot(game_level_data['PredictedRatingDifference'], kde=True, bins=30)
plt.title('Distribution of Predicted Rating Differences - Aggregated by Game')
plt.xlabel('Predicted Rating Difference')
plt.ylabel('Frequency')
plt.show()

# Visualize actual vs. predicted rating differences
plt.figure(figsize=(10, 6))
plt.scatter(game_level_data['RatingDifference'], game_level_data['PredictedRatingDifference'], alpha=0.5)
plt.plot([0, max(game_level_data['RatingDifference'].max(), game_level_data['PredictedRatingDifference'].max())],
         [0, max(game_level_data['RatingDifference'].max(), game_level_data['PredictedRatingDifference'].max())],
         'r--')  # Diagonal line for reference
plt.title('Actual vs. Predicted Rating Differences - Aggregated by Game')
plt.xlabel('Actual Rating Difference')
plt.ylabel('Predicted Rating Difference')
plt.show()

# Calculate and plot residuals
game_level_data['Residuals'] = game_level_data['RatingDifference'] - game_level_data['PredictedRatingDifference']
plt.figure(figsize=(10, 6))
sns.histplot(game_level_data['Residuals'], kde=True, bins=30)
plt.title('Residuals of Predictions - Aggregated by Game')
plt.xlabel('Residual (Actual - Predicted)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
game_level_data_sorted_desc = game_level_data.sort_values(by='PredictedRatingDifference', ascending=False).reset_index(drop=True)

In [ ]:
display(game_level_data_sorted_desc)

In [ ]:
# Set a threshold for the minimum number of occurrences required to keep the time control group
threshold = 250  # Or any other number that makes sense for your data

# Calculate the counts for each time control group
time_control_counts = new_data['TimeControl'].value_counts()

# Filter out time controls with counts below the threshold
filtered_time_controls = time_control_counts[time_control_counts >= threshold].index.tolist()

grouped = new_data.groupby('TimeControl')

# Now filter the grouped data to only include those with counts above the threshold
filtered_groups = grouped.filter(lambda x: x['TimeControl'].iloc[0] in filtered_time_controls)


# Recalculate performance metrics for the filtered groups
performance_metrics_filtered = pd.DataFrame(columns=['TimeControl', 'MAE', 'MSE', 'R2', 'Count'])

# Recalculate the metrics only for time controls above the threshold
for time_control in filtered_time_controls:
    group = filtered_groups[filtered_groups['TimeControl'] == time_control]
    mae = mean_absolute_error(group['RatingDifference'], group['PredictedRatingDifference'])
    mse = mean_squared_error(group['RatingDifference'], group['PredictedRatingDifference'])
    r2 = r2_score(group['RatingDifference'], group['PredictedRatingDifference'])
    count = time_control_counts[time_control]

    performance_metrics_filtered.loc[len(performance_metrics_filtered.index)] = [time_control, mae, mse, r2, count]

# Now, let's plot the MAE for each time control group
plt.figure(figsize=(12, 6))
sns.barplot(x='TimeControl', y='MAE', data=performance_metrics_filtered.sort_values(by='MAE', ascending=False))
plt.title('Mean Absolute Error by Time Control (Filtered)')
plt.xlabel('Time Control')
plt.ylabel('Mean Absolute Error')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# List of specific time controls we're interested in
specific_time_controls = ['60+0', '180+0', '300+0', '600+0', '1200+0']

# Filter the dataset for the specific time controls
specific_groups = new_data[new_data['TimeControl'].isin(specific_time_controls)]

# Group by the filtered 'TimeControl' and calculate performance metrics
grouped_specific = specific_groups.groupby('TimeControl')

# Initialize a DataFrame to store the performance metrics for the specific time controls
performance_metrics_specific = pd.DataFrame(columns=['TimeControl', 'MAE', 'MSE', 'R2'])

# Iterate over each group and calculate metrics
for name, group in grouped_specific:
    mae = mean_absolute_error(group['RatingDifference'], group['PredictedRatingDifference'])
    mse = mean_squared_error(group['RatingDifference'], group['PredictedRatingDifference'])
    r2 = r2_score(group['RatingDifference'], group['PredictedRatingDifference'])

    # Append the performance metrics for the current time control group
    performance_metrics_specific.loc[len(performance_metrics_specific)] = [name, mae, mse, r2]

# Ensure the 'TimeControl' column is categorical with the specified order
performance_metrics_specific['TimeControl'] = pd.Categorical(
    performance_metrics_specific['TimeControl'],
    categories=specific_time_controls,
    ordered=True
)

# Sort the DataFrame by 'TimeControl' according to the specific order we want
performance_metrics_specific.sort_values(by='TimeControl', inplace=True)

# Plotting the Mean Absolute Error for the specific time controls
plt.figure(figsize=(10, 6))
sns.barplot(x='TimeControl', y='MAE', data=performance_metrics_specific)
plt.title('Model Performance for Specific Time Controls (MAE)')
plt.xlabel('Time Control')
plt.ylabel('Mean Absolute Error')
plt.show()


In [ ]:
# List of specific time controls we're interested in
specific_time_controls = ['60+0', '180+0', '300+0', '600+0', '1200+0']

# Filter the dataset for the specific time controls
specific_groups = new_data[new_data['TimeControl'].isin(specific_time_controls)]

# Plotting predicted vs actual for each time control
fig, axes = plt.subplots(nrows=len(specific_time_controls), ncols=1, figsize=(10, 6 * len(specific_time_controls)))

for i, time_control in enumerate(specific_time_controls):
    # Filter the specific group
    group = specific_groups[specific_groups['TimeControl'] == time_control]

    # Create scatter plot for each time control
    axes[i].scatter(group['RatingDifference'], group['PredictedRatingDifference'], alpha=0.5)
    axes[i].plot([group['RatingDifference'].min(), group['RatingDifference'].max()],
                 [group['RatingDifference'].min(), group['RatingDifference'].max()],
                 'r--')  # Diagonal line for reference
    axes[i].set_title(f'Actual vs. Predicted Rating Differences for Time Control {time_control}')
    axes[i].set_xlabel('Actual Rating Difference')
    axes[i].set_ylabel('Predicted Rating Difference')
    axes[i].grid(True)

plt.tight_layout()
plt.show()
